In [175]:
import re
import requests
from lxml import html
from pymongo import MongoClient
from pprint import pprint
from datetime import datetime
import time
import json

In [3]:
header = {'Accept': '*/*',
          'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0'
}

In [9]:
session = requests.session()

In [173]:
mail_link = 'https://mail.ru/'
response = session.get(mail_link, headers=header)

In [177]:
data_ = re.findall('news:({.*})', response.text)
data = json.loads(data_[0])
result_mail = list()

for i in data['general']:
    dict_ = {}
    dict_['link'] = i['url']
    dict_['name'] = i['title'].replace('\xa0', ' ')
    dict_['source'] = 'mail.ru'
    #dict_['date'] = datetime.today().strftime("%d.%m.%Y")
    result_mail.append(dict_)
    
#print(list_url)
for i in range(len(result_mail)):
    time.sleep(10)
    res = session.get(result_mail[i]['link'], headers=header)
    print(result_mail[i]['link'])
    root = html.fromstring(res.text)
    date_ = root.xpath("//div[contains(@class, 'breadcrumbs breadcrumbs_article js-ago-wrapper')]/span/span/span[contains(@class, 'note')]/@datetime")[0]
    date_ = re.findall('[0-9]*-[0-9]*-[0-9]*', date_)[0]
    if date_:
        date = datetime.strptime(date_, '%Y-%m-%d').strftime("%d.%m.%Y")
        result_mail[i]['date'] = date
    else:
        result_mail[i]['date'] = datetime.today().strftime("%d.%m.%Y")
    
print(result_mail)

https://news.mail.ru/incident/40506320/?frommail=1
https://news.mail.ru/society/40506387/?frommail=1
https://news.mail.ru/incident/40507560/?frommail=1
https://news.mail.ru/society/40506427/?frommail=1
https://news.mail.ru/society/40505890/?frommail=1
https://news.mail.ru/society/40500004/?frommail=10
https://news.mail.ru/society/40495108/?frommail=10
https://news.mail.ru/society/40507845/?frommail=1
https://news.mail.ru/society/40507783/?frommail=1
https://news.mail.ru/incident/40505523/?frommail=1
https://news.mail.ru/politics/40504524/?frommail=1
https://sportmail.ru/news/mma/40505627/?frommail=1
https://news.mail.ru/politics/40506101/?frommail=1
[{'link': 'https://news.mail.ru/incident/40506320/?frommail=1', 'name': 'Число умерших от\xa0коронавирусной пневмонии в\xa0КНР выросло до\xa0722', 'source': 'mail.ru', 'date': '08.02.2020'}, {'link': 'https://news.mail.ru/society/40506387/?frommail=1', 'name': 'Названы вакансии с\xa0зарплатой 800 тысяч рублей', 'source': 'mail.ru', 'date': 

In [178]:
yandex_link = 'https://yandex.ru/'
response_yandex = session.get(yandex_link, headers=header)

In [184]:
root = html.fromstring(response_yandex.text)
news_href = root.xpath('//div[contains(@id,"news_panel_news")]/ol/li/a/@href')

# for yandex_news_link in news_href:
#     print(yandex_news_link)

result_yandex = list()
for yandex_news_link in news_href:
    dict_ = {}
    response_yandex_news = session.get(yandex_news_link, headers=header)
    root1 = html.fromstring(response_yandex_news.text)
    dict_['link'] = root1.xpath("//div[contains(@class, 'doc__content')]/a[contains(@class, 'link')]/@href")[0]
    dict_['name'] = root1.xpath("//h1/span[contains(@class, 'story__head-wrap')]/text()")[0]
    dict_['source'] = root1.xpath("//h1/span[contains(@class, 'story__head-agency')]/img/@alt")[0]
    dict_['date'] = datetime.today().strftime("%d.%m.%Y") #По статистике они все сегодняшние, а отправляет на разные сайты
    # в зависимости от источника (см. 'source').
    result_yandex.append(dict_)

print(result_yandex)

https://yandex.ru/news/story/Vosem_chelovek_stali_zhertvami_massovoj_draki_v_Kazakhstane--66ada6e8ed3d8609e8c475d66b11fa60?lang=ru&from=main_portal&stid=BoONclTGYYQ22iCANJPd&t=1581153530&wan=1&lr=56&msid=1581154511.21457.139851.39530&mlid=1581153530.glob_225.66ada6e8
https://yandex.ru/news/story/Rossiya_i_Belorussiya_dogovorilis_o_postavkakh_nefti_i_gaza--5a9064689e503373b3d2405c4303edba?lang=ru&from=main_portal&stid=T67puIkqCachwuRrdfAX&t=1581153530&wan=1&lr=56&msid=1581154511.21457.139851.39530&mlid=1581153530.glob_225.5a906468
https://yandex.ru/news/story/NATO_obvinilo_Rossiyu_v_presledovanii_amerikanskikh_sputnikov--f47d23fe1c4d08222e2872b5e043f190?lang=ru&from=main_portal&stid=Al5qEqIM7d1iVuXe2lQc&t=1581153530&wan=1&lr=56&msid=1581154511.21457.139851.39530&mlid=1581153530.glob_225.f47d23fe
https://yandex.ru/news/story/Nazvany_vakansii_s_zarplatoj_v_800_tysyach_rublej--b1a1250e543854eb94131bb2070c5376?lang=ru&from=main_portal&stid=COV5aW4oF0YrGjfgkmwx&t=1581153530&wan=1&lr=56&msid=

In [83]:
lenta_link = 'https://lenta.ru/parts/news/'
response_lenta = session.get(lenta_link, headers=header)

In [115]:
main_link_lenta = 'https://lenta.ru'
source = 'lenta.ru'
result_lenta = list()

root = html.fromstring(response_lenta.text)

name_ = root.xpath("//div[contains(@class, 'item news')]/div[contains(@class, 'title')]/h3/a/text()")
name = list()
for i in name_:
    name.append(i.replace('\xa0', ' '))

date_ = root.xpath("//div[contains(@class, 'item news')]/div[contains(@class, 'info g-date item__info')]/text()")
date = list()
for i in date_:
    if re.findall('[0-9]{2}:[0-9]{2}', i):
        date.append(datetime.today().strftime("%d.%m.%Y"))
    elif re.findall('[0-9]{1,2}\s[а-я]*', i):
        month_ = re.findall('[а-яА-Я]*', i)

        for m in month_:
            if m:
                month_ = m
                break

        if month_ == 'января':
            month = 1
        elif month_ == 'февраля':
            month = 2
        elif month_ == 'марта':
            month = 3
        elif month_ == 'апреля':
            month = 4
        elif month_ == 'мая':
            month = 5
        elif month_ == 'июня':
            month = 6
        elif month_ == 'июля':
            month = 7
        elif month_ == 'августа':
            month = 8
        elif month_ == 'сентября':
            month = 9
        elif month_ == 'октября':
            month = 10
        elif month_ == 'ноября':
            month = 11
        elif month_ == 'декабря':
            month = 12
        print(re.findall('[0-9]{1,2}', i))
        day = int(re.findall('[0-9]{1,2}', i)[0])

        date_time = datetime(year=2020,
                                      month=month,
                                      day=day)
        date.append(date_time.strftime("%d.%m.%Y"))
    else:
        date.append(datetime.today().strftime("%d.%m.%Y"))
        
link_ = root.xpath("//div[contains(@class, 'item news')]/div[contains(@class, 'title')]/h3/a/@href")
link = list()
for i in link_:
    link.append(main_link_lenta+i)


for i in range(len(name)):
    dict_ = {}
    dict_['link'] = link[i]
    dict_['name'] = name[i]
    dict_['source'] = source
    dict_['date'] = date[i]
    result_lenta.append(dict_)
    
for i in result_lenta:
    print(i)
    print()

{'link': 'https://lenta.ru/news/2020/02/08/ukral/', 'name': 'Освободившийся из тюрьмы россиянин обокрал любовницу и купил жене подарок', 'source': 'lenta.ru', 'date': '08.02.2020'}

{'link': 'https://lenta.ru/news/2020/02/08/kazahstandraka/', 'name': 'Восемь человек погибли в массовой драке в Казахстане', 'source': 'lenta.ru', 'date': '08.02.2020'}

{'link': 'https://lenta.ru/news/2020/02/08/duhan/', 'name': 'Гражданка США скончалась от коронавируса в Ухане', 'source': 'lenta.ru', 'date': '08.02.2020'}

{'link': 'https://lenta.ru/news/2020/02/08/su30/', 'name': 'Индия назвала устаревшими радар и комплекс РЭБ Су-30МКИ', 'source': 'lenta.ru', 'date': '08.02.2020'}

{'link': 'https://lenta.ru/news/2020/02/08/smolov/', 'name': 'Смолов нашел разницу между испанским и российским футболом', 'source': 'lenta.ru', 'date': '08.02.2020'}

{'link': 'https://lenta.ru/news/2020/02/08/versia/', 'name': 'Названа новая версия катастрофы вертолета депутата Госдумы Хайруллина', 'source': 'lenta.ru', 'dat

In [186]:
from pymongo import MongoClient
import pandas as pd

In [187]:
result = list()
result = result_mail + result_lenta + result_yandex

In [188]:
df = pd.DataFrame(result)

In [189]:
client  = MongoClient('localhost',27017)
db = client['news']
news_db = db.all

In [190]:
news_db.insert_many(result)

In [191]:
df

,link,name,source,date
0,https://news.mail.ru/incident/40506320/?fromma...,Число умерших от коронавирусной пневмонии в КН...,mail.ru,08.02.2020
1,https://news.mail.ru/society/40506387/?frommail=1,Названы вакансии с зарплатой 800 тысяч рублей,mail.ru,08.02.2020
2,https://news.mail.ru/incident/40507560/?fromma...,Восемь человек погибли в массовой драке на юге...,mail.ru,08.02.2020
3,https://news.mail.ru/society/40506427/?frommail=1,В Липецке школьникам из бедных семей запрещали...,mail.ru,08.02.2020
4,https://news.mail.ru/society/40505890/?frommail=1,Барическая пила вернется: россиян ждет резкий ...,mail.ru,08.02.2020
5,https://news.mail.ru/society/40500004/?frommai...,В Антарктиде зафиксировали новый температурный...,mail.ru,07.02.2020
6,https://news.mail.ru/society/40495108/?frommai...,Орангутанг протянул руку помощи человеку в реке,mail.ru,07.02.2020
7,https://news.mail.ru/society/40507845/?frommail=1,В чемодане сотня масок: как россияне отдыхают ...,mail.ru,08.02.2020
8,https://news.mail.ru/society/40507783/?frommail=1,Названы номинанты на звание «Худший фильм года»,mail.ru,08.02.2020
9,https://news.mail.ru/incident/40505523/?fromma...,У экс-главы минстроя Ставрополья арестовали до...,mail.ru,07.02.2020


In [192]:
news_db.remove({})

<ipython-input-192-803ae308027c>:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  news_db.remove({})


{'n': 43, 'ok': 1.0}